In [0]:
from pyspark.sql import SparkSession, functions as f
df_cpt = spark.read.parquet("/mnt/bronze/cptcodes")
df_cpt.createOrReplaceTempView('cpt_codes')
display(df_cpt)


In [0]:
%sql 
create or replace temp view  quality_check as 
select 
cpt_codes,
procedure_code_category,
procedure_code_descriptions,
code_status,
case when cpt_codes is null or procedure_code_descriptions is null then true else false end as is_quarantined
from cpt_codes;

In [0]:
%sql
select * from quality_check;

In [0]:
%sql
create table if not exists `hrcm-data-catalog`.silver.cptcodes(
  cpt_code string,
  procedure_code_category string,
  procedure_code_descriptions string,
  code_status string,
  is_quarantined boolean,
  audit_insertdate timestamp,
  audit_modifieddate timestamp,
  is_current boolean
)
using delta;

In [0]:
%sql
merge into `hrcm-data-catalog`.silver.cptcodes as target
using quality_check as source
on target.cpt_code = source.cpt_codes
and is_current = true
when matched and(
  target.procedure_code_category != source.procedure_code_category or
  target.procedure_code_descriptions != source.procedure_code_descriptions or
  target.code_status != source.code_status or
  target.is_quarantined != source.is_quarantined
)
then update 
set target.is_current = false,
    target.audit_modifieddate = current_timestamp()

In [0]:
%sql
merge into `hrcm-data-catalog`.silver.cptcodes as target
using quality_check as source
on target.cpt_code = source.cpt_codes
and is_current = true
when not matched then insert(
  target.cpt_code,
  target.procedure_code_category,
  target.procedure_code_descriptions,
  target.code_status,
  target.is_quarantined,
  target.audit_insertdate,
  target.audit_modifieddate,
  target.is_current
) values(
  source.cpt_codes,
  source.procedure_code_category,
  source.procedure_code_descriptions,
  source.code_status,
  source.is_quarantined,
  current_timestamp(),
  current_timestamp(),
  true
)

In [0]:
df_silver = spark.sql("select * from `hrcm-data-catalog`.silver.cptcodes")

df_silver.write.format("delta").mode("overwrite").save("/mnt/silver/cptcodes")